In [1]:
import os
path = os.getcwd()
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("hostname", "172.20.116.69").\
    appName("github_csv").getOrCreate()
from pyspark.sql.functions import max, desc

22/07/01 15:36:57 WARN Utils: Your hostname, DESKTOP-CHAV9A4 resolves to a loopback address: 127.0.1.1; using 172.20.116.69 instead (on interface eth0)
22/07/01 15:36:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/01 15:36:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/01 15:36:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.ml.linalg import Vectors
denseVec = Vectors.dense(1.0, 2.0, 3.0)
size = 3
idx = [1, 2] # locations of non-zero elements in vector
values = [2.0, 3.0]
sparseVec = Vectors.sparse(size, idx, values)
sparseVec.toArray()

array([0., 2., 3.])

In [2]:
df = spark.read.json("../data/simple-ml")
df.orderBy("value2").show()

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
|green| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     2|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|  red| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|green|good|     1|14.386294994851129|
|green|good|    12|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|  red|good|    35|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



In [3]:
from pyspark.ml.feature import RFormula
supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")

In [8]:
fittedRF = supervised.fit(df)
preparedDF = fittedRF.transform(df)
preparedDF.show(5)

+-----+----+------+------------------+--------------------+-----+
|color| lab|value1|            value2|            features|label|
+-----+----+------+------------------+--------------------+-----+
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|
| blue| bad|    12|14.386294994851129|(10,[2,3,6,9],[12...|  0.0|
|green|good|    15| 38.97187133755819|(10,[1,2,3,5,8],[...|  1.0|
|green|good|    12|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
+-----+----+------+------------------+--------------------+-----+
only showing top 5 rows



In the output we can see the result of our transformation—a column called features that has our
previously raw data. What’s happening behind the scenes is actually pretty simple. RFormula
inspects our data during the fit call and outputs an object that will transform our data according
to the specified formula, which is called an RFormulaModel

In [11]:
from pyspark.sql.functions import expr, col, column

In [18]:
# The first value in the features column
preparedDF.select(col('features')).take(1)[0][0].toArray()

array([ 0.        ,  1.        ,  1.        , 14.38629499,  0.        ,
        1.        ,  0.        ,  0.        , 14.38629499,  0.        ])

In [36]:
train, test = preparedDF.randomSplit([0.7, 0.3])

In [37]:
train.schema

StructType([StructField('color', StringType(), True), StructField('lab', StringType(), True), StructField('value1', LongType(), True), StructField('value2', DoubleType(), True), StructField('features', VectorUDT(), True), StructField('label', DoubleType(), False)])

# Estimator

In [38]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="label",featuresCol="features")

In [39]:
fittedLR = lr.fit(train)

In [40]:
fittedLR.transform(train).select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



# Pipeline our workflow

In [41]:
df.schema

StructType([StructField('color', StringType(), True), StructField('lab', StringType(), True), StructField('value1', LongType(), True), StructField('value2', DoubleType(), True)])

In [42]:
train, test = df.randomSplit([0.7, 0.3])

Let’s create the base stages in our pipeline. A stage simply
represents a transformer or an estimator. In our case, we will have two estimators. The RFomula
will first analyze our data to understand the types of input features and then transform them to
create new features. Subsequently, the LogisticRegression object is the algorithm that we will
train to produce a model

In [43]:
rForm = RFormula()
lr = LogisticRegression().setLabelCol("label").setFeaturesCol("features")

We will set the potential values for the RFormula in the next section. Now instead of manually
using our transformations and then tuning our model we just make them stages in the overall
pipeline, as in the following code snippet:

In [44]:
from pyspark.ml import Pipeline
stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

# Training and Evaluation

In [45]:
from pyspark.ml.tuning import ParamGridBuilder
params = ParamGridBuilder()\
  .addGrid(rForm.formula, [
    "lab ~ . + color:value1",
    "lab ~ . + color:value1 + color:value2"])\
  .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
  .addGrid(lr.regParam, [0.1, 2.0])\
  .build()

In our current paramter grid, there are three hyperparameters that will diverge from the defaults:
- Two different versions of the RFormula
- Three different options for the ElasticNet parameter
- Two different options for the regularization parameter
This gives us a total of 12 different combinations of these parameters, which means we will be
training 12 different versions of logistic regression.

The evaluator allows us to automatically and objectively compare multiple models to the same evaluation metric. We will use the BinaryClassificationEvaluator, which has a number of potential evaluation metrics. In this case we will use areaUnderROC, which is the total area under the receiver operating characteristic, a common measure of classification performance:

In [46]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()\
  .setMetricName("areaUnderROC")\
  .setRawPredictionCol("prediction")\
  .setLabelCol("label")

It is a best practice in machine learning to fit hyperparameters on a validation set (instead of your test set) to prevent overfitting. For this reason, we cannot use our holdout test set (that we created before) to tune these parameters. Luckily, Spark provides two options for performing hyperparameter tuning  automatically. We can use TrainValidationSplit, which will simply perform an arbitrary random split of our data into two different groups, or CrossValidator, which performs K-fold

In [47]:
from pyspark.ml.tuning import TrainValidationSplit
tvs = TrainValidationSplit()\
  .setTrainRatio(0.75)\
  .setEstimatorParamMaps(params)\
  .setEstimator(pipeline)\
  .setEvaluator(evaluator)

Let’s run the entire pipeline we constructed. To review, running this pipeline will test out every
version of the model against the validation set. Note the type of tvsFitted is TrainValidationSplitModel. Any time we fit a given model, it outputs a “model” type:

In [48]:
tvsFitted = tvs.fit(train)

In [49]:
# And of course evaluate how it performs on the test set!
evaluator.evaluate(tvsFitted.transform(test))

0.9047619047619048

In [66]:
tvsFitted.bestModel.stages[0].params

[Param(parent='RFormula_ff43e6162eb9', name='featuresCol', doc='features column name.'),
 Param(parent='RFormula_ff43e6162eb9', name='forceIndexLabel', doc='Force to index label whether it is numeric or string'),
 Param(parent='RFormula_ff43e6162eb9', name='formula', doc='R model formula'),
 Param(parent='RFormula_ff43e6162eb9', name='handleInvalid', doc="how to handle invalid entries. Options are 'skip' (filter out rows with invalid values), 'error' (throw an error), or 'keep' (put invalid data in a special additional bucket, at index numLabels)."),
 Param(parent='RFormula_ff43e6162eb9', name='labelCol', doc='label column name.'),
 Param(parent='RFormula_ff43e6162eb9', name='stringIndexerOrderType', doc='How to order categories of a string feature column used by StringIndexer. The last category after ordering is dropped when encoding strings. Supported options: frequencyDesc, frequencyAsc, alphabetDesc, alphabetAsc. The default value is frequencyDesc. When the ordering is set to alpha